<a href="https://colab.research.google.com/github/Saikeerthansa/Cryptocurrency-Data-Fetching-and-Analysis/blob/main/Crypto_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests gspread oauth2client pandas

In [ ]:
import requests
import pandas as pd
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets Setup
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDENTIALS_FILE = "/content/alien-lattice-442406-v8-d0966e5b3805.json" # Path to your credentials.json file
SHEET_NAME = "Live Crypto Data"

def fetch_crypto_data():
    """Fetches live cryptocurrency data for the top 50 cryptocurrencies."""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False
    }
    response = requests.get(url, params=params)
    data = response.json()
    return pd.DataFrame([{
        "Name": coin["name"],
        "Symbol": coin["symbol"],
        "Current Price (USD)": coin["current_price"],
        "Market Cap (USD)": coin["market_cap"],
        "24h Trading Volume (USD)": coin["total_volume"],
        "24h Price Change (%)": coin["price_change_percentage_24h"]
    } for coin in data])

def analyze_data(data):
    """Perform analysis on the cryptocurrency data."""
    # 1. Identifying the top 5 cryptocurrencies by market cap
    top_5_by_market_cap = data.sort_values("Market Cap (USD)", ascending=False).head(5)

    # 2. Calculating the average price of the top 50 cryptocurrencies
    average_price = data["Current Price (USD)"].mean()

    # 3. Analyzing the highest and lowest 24-hour price change
    highest_price_change = data.loc[data["24h Price Change (%)"].idxmax()]
    lowest_price_change = data.loc[data["24h Price Change (%)"].idxmin()]

    # Print the analysis results
    print("\nTop 5 Cryptocurrencies by Market Cap:")
    print(top_5_by_market_cap[["Name", "Market Cap (USD)"]])

    print("\nAverage Price of Top 50 Cryptocurrencies (USD):")
    print(f"${average_price:.2f}")

    print("\nCryptocurrency with the Highest 24h Price Change:")
    print(f"{highest_price_change['Name']} ({highest_price_change['Symbol']}): {highest_price_change['24h Price Change (%)']:.2f}%")

    print("\nCryptocurrency with the Lowest 24h Price Change:")
    print(f"{lowest_price_change['Name']} ({lowest_price_change['Symbol']}): {lowest_price_change['24h Price Change (%)']:.2f}%")

def update_google_sheet(data):
    """Updates the Google Sheet with the live cryptocurrency data."""
    # Authenticate using service account credentials
    credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, SCOPE)
    client = gspread.authorize(credentials)
    sheet = client.open(SHEET_NAME).sheet1  # Open the first sheet in the document

    # Clear existing data and update with new data
    sheet.clear()  # Clears the existing data
    sheet.update([data.columns.values.tolist()] + data.values.tolist())  # Updates new data
    print("Google Sheet updated successfully.")

# Fetch and update data every 5 minutes
def run_script():
    try:
        while True:
            # Fetch live crypto data
            crypto_data = fetch_crypto_data()

            # Perform the analysis
            analyze_data(crypto_data)

            # Ensure data is not empty and update Google Sheet
            if not crypto_data.empty:
                update_google_sheet(crypto_data)

            # Sleep for 5 minutes (300 seconds)
            time.sleep(300)  # Wait for 5 minutes before fetching the next data
    except KeyboardInterrupt:
        print("\nLive updating stopped by user.")

if __name__ == "__main__":
    run_script()



Top 5 Cryptocurrencies by Market Cap:
       Name  Market Cap (USD)
0   Bitcoin     1921752189980
1  Ethereum      376784099079
2    Tether      130154219009
3    Solana      115053251317
4       BNB       89164409098

Average Price of Top 50 Cryptocurrencies (USD):
$4257.72

Cryptocurrency with the Highest 24h Price Change:
Bitcoin Cash (bch): 19.79%

Cryptocurrency with the Lowest 24h Price Change:
MANTRA (om): -10.55%
Google Sheet updated successfully.
